In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open("tags/recipe_output.txt", encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

corpus length: 10043373


In [3]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(512, input_shape=(maxlen, len(chars)), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(512))
model.add(Dropout(0.2))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

from keras.callbacks import ModelCheckpoint

filepath="weights-improvement-{epoch:02d}-{loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='auto')

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
callbacks_list = [print_callback, checkpoint]

total chars: 130
nb sequences: 3347778
Vectorization...
Build model...


In [4]:
model.fit(x, y,
          batch_size=128,
          epochs=300,
          callbacks=callbacks_list)

Epoch 1/300
3347778/3347778 [==============================] - 37171s 11ms/step - loss: 3.2944

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "ecipe: penne and cauliflower with mustar"
ecipe: penne and cauliflower with mustaraiu utaiuiuta

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: RuntimeWarning: divide by zero encountered in log


 uuautuaaiauiittuuaaauiaaaaieiiauuauiiautaaaataituaitauiaaaauitiaaaiatuutuuuuuaautattataautaaiiiatttauuuuttauuiueaauuuii uuauaauauiuaaaauataaiaiuuautauaaaateaauuuautiaitataatauaiiaataatauutaitiautuuataiutuitautiuitaiaatuatiaiutiattataaatautaouuitauituaitauttaaituiaautaaia auaaaauaaieuaitttaaaauutiuataauuatuuuaiiaauuaituiaaaatutaituttiataauuuiiautaautiuua aiuitiaiatatiuauitiuautuiatuti
----- diversity: 0.5
----- Generating with seed: "ecipe: penne and cauliflower with mustar"
ecipe: penne and cauliflower with mustariteu,uaiiiiaeu aaiiauiiaato  iet,iuauiitu itteuiaanatiaaabtnuuuiiutitiitot i,eauoaiaiiiitetiutuitai aatuatiitauiaaaa t oaa,aielaaiiaia.naiii,ai ues,t,utuaiaudaaeeiai,uatetteieumuaataioatuaaauea)eaaauatuiuitittaa aiiaitueue,aauuttitdaiutiuaa taioieutattiutuvitiaatau tianoeuuuti teiateuaiaii, aue,  ta aaaiiii  laetuadku uauiaatutir.teuuiuitaeuka ura itaiuuttaat eeiu tiaeaaatitu aiidiitiitu ueaeaaese
----- diversity: 1.0
----- Generating with seed: "ecipe: penne and cauliflower 

KeyboardInterrupt: 